<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/test_simulate_and_analyze_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pytest
from simulate_and_analyze import (
    simulate, simulate_with_noise,
    summary_statistics, fit_regression_slope
)

def test_simulate_length_and_columns():
    df = simulate(1000, 10.0, 0.0, 0.1, breach_step=500, breach_value=1.0)
    assert len(df) == 1000
    assert set(df.columns) == {'step', 'log_output', 'log_beyond_field'}
    assert df.loc[499, 'log_beyond_field'] == pytest.approx(1.0)

def test_simulate_with_noise_reproducible():
    df = simulate(500, 5.0, 0.0, 0.2, breach_step=250, breach_value=0.5)
    np.random.seed(0)
    noisy1 = simulate_with_noise(df, noise_sd=0.0)
    np.random.seed(0)
    noisy2 = simulate_with_noise(df, noise_sd=0.0)
    # With zero noise, results must match exactly
    pd.testing.assert_series_equal(noisy1['log_output_noisy'], noisy2['log_output_noisy'])

def test_summary_statistics_correctness():
    df = pd.DataFrame({'val': [1, 2, 3, 4, 5]})
    stats = summary_statistics(df, 'val')
    assert stats['mean'] == 3.0
    assert pytest.approx(stats['variance'], rel=1e-6) == 2.0
    assert stats['min'] == 1
    assert stats['max'] == 5

def test_fit_regression_slope_linear():
    # y = 2*x + 1
    x = np.arange(0, 100).reshape(-1, 1)
    y = 2.0 * x.flatten() + 1.0
    df = pd.DataFrame({'step': x.flatten(), 'out': y})
    slope = fit_regression_slope(df, 'step', 'out')
    assert pytest.approx(slope, rel=1e-6) == 2.0